In [212]:
title = 'AMP4_B06B viability'
#file with a resazurin fluorescence measurements (table; Excel format)
readout_file_Omega_xlsx = 'input/viability/AMP04-B06B viability.xlsx'
#file with a layout (table; Excel format) in a sheet titled 'Layout'
layout_file_xlsx = 'input/viability/AMP04-B06B viability.xlsx'

In [213]:
import pandas as pd
import numpy as np
import warnings,re
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import leastsq
from matplotlib.ticker import ScalarFormatter
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio
import plotly.figure_factory as ff
import plotly.express as px
import seaborn as sns
from pandas.api.types import CategoricalDtype
from matplotlib import rcParams
import matplotlib.font_manager as font_manager
warnings.filterwarnings("ignore"); pd.options.display.width = 5000; pio.renderers.default = 'iframe'
pd.set_option('display.max_rows', 999)

#Adding custom font
font_path = 'input/Roboto/'
font_files = font_manager.findSystemFonts(fontpaths=font_path)
font_list = font_manager.createFontList(font_files)
font_manager.fontManager.ttflist.extend(font_list)
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = 'Roboto'

In [214]:
data = pd.read_excel(readout_file_Omega_xlsx,sheet_name='Microplate End point',skiprows=14,index_col=[0])
data = data.apply(pd.to_numeric,errors='coerce')
layout = pd.read_excel(layout_file_xlsx,sheet_name='Layout',index_col=[0])
layout

,1,2,3,4,5,6,7,8,9,10,11,12
A,NaN,DMSO,DMSO,DMSO,DMSO,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B,NaN,FSK 20,FSK 10,FSK 5,FSK 2.5,FSK 1.25,FSK 0.625,FSK 0.3125,FSK 0.15625,FSK 0.078125,FSK 0,NaN
C,NaN,FSK 20,FSK 10,FSK 5,FSK 2.5,FSK 1.25,FSK 0.625,FSK 0.3125,FSK 0.15625,FSK 0.078125,FSK 0,NaN
D,NaN,FSK 20,FSK 10,FSK 5,FSK 2.5,FSK 1.25,FSK 0.625,FSK 0.3125,FSK 0.15625,FSK 0.078125,FSK 0,NaN
E,NaN,FTD 20,FTD 10,FTD 5,FTD 2.5,FTD 1.25,FTD 0.625,FTD 0.3125,FTD 0.15625,FTD 0.078125,FTD 0,NaN
F,NaN,FTD 20,FTD 10,FTD 5,FTD 2.5,FTD 1.25,FTD 0.625,FTD 0.3125,FTD 0.15625,FTD 0.078125,FTD 0,NaN
G,NaN,FTD 20,FTD 10,FTD 5,FTD 2.5,FTD 1.25,FTD 0.625,FTD 0.3125,FTD 0.15625,FTD 0.078125,FTD 0,NaN
H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [215]:
minimum = 'A2:A5'
regex_min = re.findall(r'[A-Z]{1}[0-9]{1,2}',minimum.upper())
regex_min = [[ord(x[0])-65,int(x[1:])-1] for x in regex_min]
min_frame = data.iloc[regex_min[0][0]:regex_min[1][0]+1,regex_min[0][1]:regex_min[1][1]+1]
layout.iloc[regex_min[0][0]:regex_min[1][0]+1,regex_min[0][1]:regex_min[1][1]+1]

,2,3,4,5
A,DMSO,DMSO,DMSO,DMSO


In [216]:
avg_min = np.mean(min_frame.values)
avg_min

37291.0

In [217]:
maximum = 'B11:G11'
regex_max = re.findall(r'[A-Z]{1}[0-9]{1,2}',maximum.upper())
regex_max = [[ord(x[0])-65,int(x[1:])-1] for x in regex_max]
max_frame = data.iloc[regex_max[0][0]:regex_max[1][0]+1,regex_max[0][1]:regex_max[1][1]+1]
layout.iloc[regex_max[0][0]:regex_max[1][0]+1,regex_max[0][1]:regex_max[1][1]+1]

,11
B,FSK 0
C,FSK 0
D,FSK 0
E,FTD 0
F,FTD 0
G,FTD 0


In [218]:
avg_max = np.mean(max_frame.values)
avg_max

195642.5

In [219]:
def min_max(x,minimum,maximum):
    x_ = (float(x)-minimum)/(maximum-minimum)
    return x_*100

result = data.applymap(lambda x: min_max(x, minimum=avg_min,maximum=avg_max))
result

,1,2,3,4,5,6,7,8,9,10,11,12
Fluorescence (FI),,,,,,,,,,,,
A,NaN,-0.319542,-0.015156,-0.164192,0.498890,-23.343637,-23.355636,-23.368266,-23.369529,-23.341111,-23.210390,NaN
B,NaN,102.143017,98.081799,93.814078,89.082832,89.774963,96.283269,92.551065,93.403599,96.345156,95.904996,NaN
C,NaN,100.787173,102.842095,98.230203,97.378301,100.969047,99.233667,99.351759,97.711736,98.371029,98.384922,NaN
D,NaN,102.330575,101.791268,101.678860,102.542129,102.633066,104.768190,101.707909,98.599003,101.121240,100.083675,NaN
E,NaN,100.754966,102.601491,105.383909,106.845215,104.390549,104.135420,103.054913,103.968703,103.322040,104.403179,NaN
F,NaN,97.550071,96.477141,101.774217,105.108572,101.436993,105.173617,103.656738,102.891984,102.488451,97.968128,NaN
G,NaN,99.865173,99.996527,102.208062,105.660508,107.495035,104.686094,107.163494,105.897955,107.258220,103.255100,NaN
H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [220]:
layout

,1,2,3,4,5,6,7,8,9,10,11,12
A,NaN,DMSO,DMSO,DMSO,DMSO,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B,NaN,FSK 20,FSK 10,FSK 5,FSK 2.5,FSK 1.25,FSK 0.625,FSK 0.3125,FSK 0.15625,FSK 0.078125,FSK 0,NaN
C,NaN,FSK 20,FSK 10,FSK 5,FSK 2.5,FSK 1.25,FSK 0.625,FSK 0.3125,FSK 0.15625,FSK 0.078125,FSK 0,NaN
D,NaN,FSK 20,FSK 10,FSK 5,FSK 2.5,FSK 1.25,FSK 0.625,FSK 0.3125,FSK 0.15625,FSK 0.078125,FSK 0,NaN
E,NaN,FTD 20,FTD 10,FTD 5,FTD 2.5,FTD 1.25,FTD 0.625,FTD 0.3125,FTD 0.15625,FTD 0.078125,FTD 0,NaN
F,NaN,FTD 20,FTD 10,FTD 5,FTD 2.5,FTD 1.25,FTD 0.625,FTD 0.3125,FTD 0.15625,FTD 0.078125,FTD 0,NaN
G,NaN,FTD 20,FTD 10,FTD 5,FTD 2.5,FTD 1.25,FTD 0.625,FTD 0.3125,FTD 0.15625,FTD 0.078125,FTD 0,NaN
H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [221]:
output = title+'.xlsx'
writer = pd.ExcelWriter(f'output/viability/{output}',engine='xlsxwriter')
result.to_excel(writer,sheet_name='Results - concentration',startrow=1, startcol=0)
layout.to_excel(writer,sheet_name='Results - concentration',startrow=12, startcol=0)
writer.save()